# Model training with +ve Xeff

## Load data

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model


In [15]:
outdir = "out_positive_xeff"
# import data
data = pd.read_pickle(f"{outdir}/p_a1_positive_xeff.pkl")

In [16]:
data

,q,xeff,a1,p_a1
0,0.1,0.1,"[0.012024048096192383, 0.014003959823454523, 0...","[0.009216418726310445, 0.028458441503637077, 0..."
1,0.1,0.2,"[0.1222444889779559, 0.12400351805816039, 0.12...","[0.0013008215791223796, 0.004041149522956928, ..."
2,0.1,0.3,"[0.23246492985971942, 0.23400307629286626, 0.2...","[0.0010697856954987733, 0.0024382824268816272,..."
3,0.1,0.4,"[0.3426853707414829, 0.3440026345275721, 0.345...","[0.0009629691381983512, 0.00222891654232283, 0..."
4,0.1,0.5,"[0.45290581162324645, 0.454002192762278, 0.455...","[0.001315515963205737, 0.0027549382395791115, ..."
...,...,...,...,...
85,1.0,0.5,"[0.002004008016032064, 0.004003999983935807, 0...","[0.004842086019793161, 0.00951858515585301, 0...."
86,1.0,0.6,"[0.20240480961923846, 0.2040031967743101, 0.20...","[0.0010434286955042496, 0.0013208399318507351,..."
87,1.0,0.7,"[0.40280561122244485, 0.4040023935646844, 0.40...","[0.0018007422164230593, 0.0023384387900644075,..."
88,1.0,0.8,"[0.6032064128256512, 0.6040015903550587, 0.604...","[0.0032252396366210193, 0.004504994041532742, ..."


## Pre-processing

### Some data contains nans, drop them so that they wont cause problems later

In [17]:
# data.drop([0,1,2,3,4,5,6,7,8,9,10,11,21,22,32,33,43,44,54,55,65,66,76,77,87,88,98,99,109,110,120], inplace=True)
# data.reset_index(drop=True, inplace=True)

### Stupid way of getting everything to  numpy arrays with dtyoe = float64

### Spliting data into train and test

In [18]:
# # a1
# a1 = []
# for i in range(len(data)):
#     a1.append(data['a1'][i])
# a1 = np.array(a1)
# # p
# p = []
# for i in range(len(data)):
#     p.append(data['p_a1'][i])
# p = np.array(p)
# # q
# q = []
# for i in data['q'].values:
#     for j in range(len(a1[0])):
#         q.append(i)
# q = np.array(q)
# # xeff 
# xeff = []
# for i in data['xeff'].values:
#     for j in range(len(a1[0])):
#         xeff.append(i)
# xeff = np.array(xeff)

# a1 = a1.flatten()
# p = p.flatten()

# print(xeff.shape, q.shape, p.shape, a1.shape)

Nice

## Split data into train and test

In [19]:
# split into train and test
from sklearn.model_selection import train_test_split
train_q, test_q, train_xeff, test_xeff, train_p, test_p, train_a1, test_a1 \
= train_test_split(q, xeff, p, a1,
 test_size=0.2, random_state=42)

In [20]:
train_q_xeff =  np.stack((train_q, train_xeff), axis=1)

## Define evoluation function

In [21]:
def eva_model(model):
    # evaluate the model
    global test_a1
    global test_p
    scores = model.evaluate([test_q,test_xeff, test_a1], test_p)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    # plot history
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    #
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['train', 'val'], loc='upper left')
    plt.xlabel('epoches')
    plt.ylabel('loss')
    plt.title('model loss')
    plt.show()
    #
    # test_p_pred = model.predict([test_q,test_xeff, test_a1])
    # # test_p_pred = scaler_p.inverse_transform(test_p_pred)
    # # test_p = scaler_p.inverse_transform(test_p)
    # # test_a1 = scaler_a.inverse_transform(test_a1)
    # plt.plot(test_a1[3],test_p_pred[3],label = 'pred',color = 'red')
    # plt.plot(test_a1[3],test_p[3],label = 'true',color = 'blue')
    # plt.title(f'prediction vs true p_a1 for q = {test_q[3]}, xeff = {test_xeff[3]}')
    # plt.legend()
    # plt.xlabel('a1')
    # plt.ylabel('p')
    # plt.show()

    # plt.plot(test_a1[8],test_p_pred[8],label = 'pred',color = 'red')
    # plt.plot(test_a1[8],test_p[8],label = 'true',color = 'blue')
    # plt.title(f'prediction vs true p_a1 for q = {test_q[5]}, xeff = {test_xeff[5]}')
    # plt.legend()
    # plt.xlabel('a1')
    # plt.ylabel('p')
    # plt.show()

## Define Model

In [22]:
Input1 = keras.Input(shape=(1,)) # q
Input2 = keras.Input(shape=(1,)) # xeff
Input3 = keras.Input(shape=(1,)) # a1
merged = keras.layers.concatenate([Input1, Input2,Input3]) # Merging Laryer
# 
x = keras.layers.Dense(32, activation='relu',
 kernel_regularizer= keras.regularizers.L2(0.01))(merged) # Fully Connected Layer
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization()(x)
#
output = keras.layers.Dense(1)(x) # Output
#
model = Model(inputs=[Input1,Input2,Input3], outputs=output)
#
# opt = keras.optimizers.Adam(learning_rate = 3e-3)
model.compile(optimizer= 'adam',
                loss='mse',
                metrics= ['accuracy'])
#
history = model.fit([train_q,train_xeff,train_a1], train_p, validation_split=0.2,
                    epochs=5000, batch_size=32, shuffle=True, verbose=2)

Epoch 1/5000
900/900 - 1s - loss: 5.3131 - accuracy: 0.0000e+00 - val_loss: 2.7768 - val_accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 2/5000
900/900 - 1s - loss: 2.4679 - accuracy: 0.0000e+00 - val_loss: 1.2859 - val_accuracy: 0.0000e+00 - 841ms/epoch - 934us/step
Epoch 3/5000
900/900 - 1s - loss: 1.8240 - accuracy: 0.0000e+00 - val_loss: 0.9451 - val_accuracy: 0.0000e+00 - 771ms/epoch - 857us/step
Epoch 4/5000
900/900 - 1s - loss: 1.4230 - accuracy: 0.0000e+00 - val_loss: 0.7147 - val_accuracy: 0.0000e+00 - 849ms/epoch - 943us/step
Epoch 5/5000
900/900 - 1s - loss: 1.2982 - accuracy: 0.0000e+00 - val_loss: 0.7095 - val_accuracy: 0.0000e+00 - 711ms/epoch - 790us/step
Epoch 6/5000
900/900 - 1s - loss: 1.2267 - accuracy: 0.0000e+00 - val_loss: 0.4819 - val_accuracy: 0.0000e+00 - 703ms/epoch - 781us/step
Epoch 7/5000
900/900 - 1s - loss: 1.1460 - accuracy: 0.0000e+00 - val_loss: 0.4297 - val_accuracy: 0.0000e+00 - 729ms/epoch - 810us/step
Epoch 8/5000
900/900 - 1s - loss: 1.0836 - acc

KeyboardInterrupt: 